<div class="alert alert-block alert-info">
<p style="font-size:24px;text-align:center"><b>Single crystal pass</b>
</div>

In [ ]:
# Laser pulse
set = 'function' # Options: 'old', 'new', 'function'
num_laser_slices = 10
pulse_ncells = 128

# Crystal
prop_type = 'n0n2_srw' # Options: 'gain_calc' 'abcd_lct' 'n0n2_lct' 'n0n2_srw'
pump_type = 'dual' # Options: 'dual' 'left' 'right'
num_crystal_slices = 10
gain = 1
radial_n2 = 1

# Drift
L_drift = 0.1 #[m]


### Imports, Files, Crystal/Laser Params

In [ ]:
import sys, copy
import numpy as np
from pykern.pkcollections import PKDict
import os

# The rslaser library may not be installed, so a check is required.
try:
    import rslaser
except:
    # Developers should use 'pip install -e .' from the command line.
    # Users can install directly from GitHub --
    !{sys.executable} -m pip install git+https://github.com/radiasoft/rslaser.git
    import rslaser

from rslaser.pulse import pulse
from rslaser.optics import crystal
from rslaser.optics import drift
import rslaser.utils.srwl_uti_data as srwutil
import uti_plot

import srwlib

import scipy.constants as const
from scipy import interpolate
from scipy import special

# 2D plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm

# reset the notebook style
mpl.rcParams.update(mpl.rcParamsDefault)
plt.rcParams['pcolor.shading'] ='auto'
%matplotlib inline

In [ ]:
if set == 'old':
    package_data_dir = rslaser.pkg_resources.resource_filename ('rslaser', 'package_data/20220218')
    ccd_name = 'photon_count_pump_off.txt'
    wfs_name = 'phase_pump_off.txt'
elif set == 'new':
    package_data_dir = rslaser.pkg_resources.resource_filename ('rslaser', 'package_data/20220519')
    ccd_name = 'cut_photon_count_seed_laser.txt'
    wfs_name = 'phase_pump_off_zeroed.txt'    

if set != 'function':
    files = PKDict(
        meta = os.path.join(package_data_dir, 'meta_data.dat'),
        ccd = os.path.join(package_data_dir, ccd_name),
        wfs = os.path.join(package_data_dir, wfs_name)
    )

z       = np.array([ 0.0,  0.5,  1.0, 1.5, 2.0, 2.5])/100.0
n2_plot = np.array([30.5, 18.3, 10.4, 5.9, 3.3, 1.9])
plot_fit = interpolate.splrep(z, n2_plot)

In [ ]:
w0 = 1.64e-3

params = PKDict(
    photon_e_ev = 1.5498, # Photon energy [eV], calculated from 800nm wavelength
    nslice      = num_laser_slices,
    pulseE      = 1.0e-6,
    tau_fwhm    = 300.0e-12 / np.sqrt(2.),
    sigx_waist  = w0,
    sigy_waist  = w0,
    nx_slice    = pulse_ncells,
    ny_slice    = pulse_ncells,
)

L_cryst = 0.025
new_z = (L_cryst /num_crystal_slices) *(np.arange(num_crystal_slices)+0.5)

crystal_params = PKDict(
    length      = L_cryst, # [m]
    nslice      = num_crystal_slices,
    n0          = [1.76 for _ in range(num_crystal_slices)],
    n2          = interpolate.splev(new_z, plot_fit).tolist(),
    l_scale     = 0.001,
    population_inversion = PKDict(
        n_cells=64,
        mesh_extent=0.01,  # [m]
        crystal_alpha=120.0,  # [1/m], 1.2 1/cm
        pump_waist=1.43e-3,  # [m]
        pump_wavelength=532.0e-9,  # [m]
        pump_energy=0.035,  # [J], pump laser energy onto the crystal
        pump_type=pump_type,
    )
)

cell_dx = (2.0 *crystal_params.population_inversion.mesh_extent)/crystal_params.population_inversion.n_cells
cell_volume = cell_dx**2.0 *(crystal_params.length /crystal_params.nslice)

***
## Initial Intensity and Phase

In [ ]:
# Initialize laser pulse
if set == 'function':
    thisPulse = pulse.LaserPulse(params)
else:
    thisPulse = pulse.LaserPulse(params, files)

thisPulse.zero_phase()

# Plot wavefront
lp_wfr_x = np.linspace(thisPulse.slice[0].wfr.mesh.xStart,thisPulse.slice[0].wfr.mesh.xFin,thisPulse.slice[0].wfr.mesh.nx)
lp_wfr_y = np.linspace(thisPulse.slice[0].wfr.mesh.yStart,thisPulse.slice[0].wfr.mesh.yFin,thisPulse.slice[0].wfr.mesh.ny)

In [ ]:
e_total_init = thisPulse.extract_total_2d_elec_fields()
intens_2d_init = 0.5 *const.c *const.epsilon_0 *(e_total_init.re**2.0 + e_total_init.im**2.0)

intens0 = intens_2d_init.flatten()
wfr0 = thisPulse.slice[0].wfr
#srwlib.srwl_uti_save_intens_ascii(intens0, wfr0.mesh, 'initial-intensity.dat', 0, ['', 'Horizontal Position', 'Vertical Position', 'Intensity'], _arUnits=['', 'm', 'm', ''])
uti_plot.uti_plot2d1d(
    intens0,
    [wfr0.mesh.xStart, wfr0.mesh.xFin, wfr0.mesh.nx],
    [wfr0.mesh.yStart, wfr0.mesh.yFin, wfr0.mesh.ny],
    0,
    0,
    ['Horizontal Position', 'Vertical Position', 'Intensity (initial)'],
    ['m', 'm', 'ph/s/.1%bw/mm^2'],
    True)

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(4.6 * 2,3.6 * 2))
    ax = fig.gca()
    plt.pcolormesh(lp_wfr_x*(1e3), lp_wfr_y*(1e3), intens_2d_init, cmap=plt.cm.viridis, shading='auto')
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Intensity (initial)')

In [ ]:
phase_2d_init = thisPulse.extract_total_2d_phase()
phase0 = phase_2d_init.flatten()
wfr0 = thisPulse.slice[0].wfr
#srwlib.srwl_uti_save_intens_ascii(phase0, wfr0.mesh, 'initial-phase.dat', 0, ['', 'Horizontal Position', 'Vertical Position', 'Phase'], _arUnits=['', 'm', 'm', 'rad'])
uti_plot.uti_plot2d1d(
    phase0,
    [wfr0.mesh.xStart, wfr0.mesh.xFin, wfr0.mesh.nx],
    [wfr0.mesh.yStart, wfr0.mesh.yFin, wfr0.mesh.ny],
    0,
    0,
    ['Horizontal Position', 'Vertical Position', 'Phase (initial)'],
    ['m', 'm', ''],
    True)

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(4.6 * 2,3.6 * 2))
    ax = fig.gca()
    plt.pcolormesh(lp_wfr_x*(1e3), lp_wfr_y*(1e3), phase_2d_init, cmap=plt.cm.viridis, shading='auto')
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Phase (initial)')

In [ ]:
rayleigh_length_init = np.pi * (thisPulse.sigx_waist)**2.0 / (thisPulse._lambda0)
print('Rayleigh Length:', round(rayleigh_length_init,3), ' m')
print('RMS bunch length:', round(thisPulse.sig_s,3), ' m')

***
## Propagate through crystal once

In [ ]:
# Initialize laser pulse
if set == 'function':
    thisPulse = pulse.LaserPulse(params)
else:
    thisPulse = pulse.LaserPulse(params, files)

thisPulse.zero_phase()
e_crystal = crystal.Crystal(crystal_params)
e_drift = drift.Drift(L_drift)

n_photons_before = 0
for laser_index_i in np.arange(thisPulse.nslice):
        n_photons_before += np.sum(thisPulse.slice[laser_index_i].n_photons_2d.mesh)

print('\nPropagating ', thisPulse.nslice, 'laser slices through ', e_crystal.nslice,' crystal slices')
thisPulse = e_crystal.propagate(thisPulse, prop_type, gain, radial_n2)

x_prop = np.linspace(thisPulse.slice[0].wfr.mesh.xStart,thisPulse.slice[0].wfr.mesh.xFin,thisPulse.slice[0].wfr.mesh.nx)
y_prop = np.linspace(thisPulse.slice[0].wfr.mesh.yStart,thisPulse.slice[0].wfr.mesh.yFin,thisPulse.slice[0].wfr.mesh.ny)

In [ ]:
e_total_prop = thisPulse.extract_total_2d_elec_fields()
intens_2d_prop = 0.5 *const.c *const.epsilon_0 *(e_total_prop.re**2.0 + e_total_prop.im**2.0)

intens0 = intens_2d_prop.flatten()
wfr0 = thisPulse.slice[0].wfr
#srwlib.srwl_uti_save_intens_ascii(intens0, wfr0.mesh, 'initial-intensity.dat', 0, ['', 'Horizontal Position', 'Vertical Position', 'Intensity'], _arUnits=['', 'm', 'm', ''])
uti_plot.uti_plot2d1d(
    intens0,
    [wfr0.mesh.xStart, wfr0.mesh.xFin, wfr0.mesh.nx],
    [wfr0.mesh.yStart, wfr0.mesh.yFin, wfr0.mesh.ny],
    0,
    0,
    ['Horizontal Position', 'Vertical Position', 'Intensity (propagate)'],
    ['m', 'm', 'ph/s/.1%bw/mm^2'],
    True)

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(4.6 * 2,3.6 * 2))
    ax = fig.gca()
    plt.pcolormesh(x_prop*(1e3), y_prop*(1e3), intens_2d_prop, cmap=plt.cm.viridis, shading='auto')
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Intensity (propagate)')

In [ ]:
phase_2d_prop = thisPulse.extract_total_2d_phase()
phase0 = phase_2d_prop.flatten()
wfr0 = thisPulse.slice[0].wfr
#srwlib.srwl_uti_save_intens_ascii(phase0, wfr0.mesh, 'initial-phase.dat', 0, ['', 'Horizontal Position', 'Vertical Position', 'Phase'], _arUnits=['', 'm', 'm', 'rad'])
uti_plot.uti_plot2d1d(
    phase0,
    [wfr0.mesh.xStart, wfr0.mesh.xFin, wfr0.mesh.nx],
    [wfr0.mesh.yStart, wfr0.mesh.yFin, wfr0.mesh.ny],
    0,
    0,
    ['Horizontal Position', 'Vertical Position', 'Phase (propagate)'],
    ['m', 'm', ''],
    True)

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(4.6 * 2,3.6 * 2))
    ax = fig.gca()
    plt.pcolormesh(x_prop*(1e3), y_prop*(1e3), phase_2d_prop, cmap=plt.cm.viridis, shading='auto')
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Phase (propagate)')

In [ ]:
print('Drifting ', thisPulse.nslice, 'laser slices by ', L_drift, '[m]')
thisPulse = e_drift.propagate(thisPulse, 'default')

x_prop = np.linspace(thisPulse.slice[0].wfr.mesh.xStart,thisPulse.slice[0].wfr.mesh.xFin,thisPulse.slice[0].wfr.mesh.nx)
y_prop = np.linspace(thisPulse.slice[0].wfr.mesh.yStart,thisPulse.slice[0].wfr.mesh.yFin,thisPulse.slice[0].wfr.mesh.ny)

In [ ]:
e_total_prop = thisPulse.extract_total_2d_elec_fields()
intens_2d_prop = 0.5 *const.c *const.epsilon_0 *(e_total_prop.re**2.0 + e_total_prop.im**2.0)

intens0 = intens_2d_prop.flatten()
wfr0 = thisPulse.slice[0].wfr
#srwlib.srwl_uti_save_intens_ascii(intens0, wfr0.mesh, 'initial-intensity.dat', 0, ['', 'Horizontal Position', 'Vertical Position', 'Intensity'], _arUnits=['', 'm', 'm', ''])
uti_plot.uti_plot2d1d(
    intens0,
    [wfr0.mesh.xStart, wfr0.mesh.xFin, wfr0.mesh.nx],
    [wfr0.mesh.yStart, wfr0.mesh.yFin, wfr0.mesh.ny],
    0,
    0,
    ['Horizontal Position', 'Vertical Position', 'Intensity (drift)'],
    ['m', 'm', 'ph/s/.1%bw/mm^2'],
    True)

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(4.6 * 2,3.6 * 2))
    ax = fig.gca()
    plt.pcolormesh(x_prop*(1e3), y_prop*(1e3), intens_2d_prop, cmap=plt.cm.viridis, shading='auto')
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Intensity (drift)')

In [ ]:
phase_2d_prop = thisPulse.extract_total_2d_phase()
phase0 = phase_2d_prop.flatten()
wfr0 = thisPulse.slice[0].wfr
#srwlib.srwl_uti_save_intens_ascii(phase0, wfr0.mesh, 'initial-phase.dat', 0, ['', 'Horizontal Position', 'Vertical Position', 'Phase'], _arUnits=['', 'm', 'm', 'rad'])
uti_plot.uti_plot2d1d(
    phase0,
    [wfr0.mesh.xStart, wfr0.mesh.xFin, wfr0.mesh.nx],
    [wfr0.mesh.yStart, wfr0.mesh.yFin, wfr0.mesh.ny],
    0,
    0,
    ['Horizontal Position', 'Vertical Position', 'Phase (drift)'],
    ['m', 'm', ''],
    True)

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(4.6 * 2,3.6 * 2))
    ax = fig.gca()
    plt.pcolormesh(x_prop*(1e3), y_prop*(1e3), phase_2d_prop, cmap=plt.cm.viridis, shading='auto')
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Phase (drift)')

In [ ]:
prop_n_photons = 0
for laser_index_i in np.arange(thisPulse.nslice):
        prop_n_photons += np.sum(thisPulse.slice[laser_index_i].n_photons_2d.mesh)

# print(prop_n_photons -n_photons_before)
print('Initial number: ', "{:.2E}".format(n_photons_before))
print('Propagated number (from array):', "{:.2E}".format(prop_n_photons))

In [ ]:
total = 0
for laser_index_i in np.arange(thisPulse.nslice):
    
    dx = (thisPulse.slice[laser_index_i].wfr.mesh.xFin - thisPulse.slice[laser_index_i].wfr.mesh.xStart)/thisPulse.slice[laser_index_i].wfr.mesh.nx
    dy = (thisPulse.slice[laser_index_i].wfr.mesh.yFin - thisPulse.slice[laser_index_i].wfr.mesh.yStart)/thisPulse.slice[laser_index_i].wfr.mesh.ny
    cell_area = dx * dy
    
    intens_2d = srwutil.calc_int_from_elec(thisPulse.slice[laser_index_i].wfr) #extract 2d intensity
    efield_abs_sqrd_2d = np.sqrt(const.mu_0 / const.epsilon_0) * 2.0 * intens_2d # [V^2/m^2]
    
    end1 = (thisPulse.slice[laser_index_i]._pulse_pos -0.5*thisPulse.slice[laser_index_i].ds) /(np.sqrt(2.0) *thisPulse.slice[laser_index_i].sig_s)
    end2 = (thisPulse.slice[laser_index_i]._pulse_pos +0.5*thisPulse.slice[laser_index_i].ds) /(np.sqrt(2.0) *thisPulse.slice[laser_index_i].sig_s)
    energy_2d = cell_area *(const.epsilon_0 /2.0) \
                *(efield_abs_sqrd_2d /np.exp(-thisPulse.slice[laser_index_i]._pulse_pos**2.0/(np.sqrt(2.0) *thisPulse.slice[laser_index_i].sig_s)**2.0)) \
                *((np.sqrt(np.pi)/2.0) *(np.sqrt(2.0) *thisPulse.slice[laser_index_i].sig_s) *(special.erf(end2) -special.erf(end1)))
    photon_e = thisPulse.slice[laser_index_i].photon_e_ev * const.e
    n_photons_2d = energy_2d / photon_e
    total += np.sum(n_photons_2d)

print('Propagated number (from fields):', "{:.2E}".format(total))